In [ ]:
#a)
import pandas as pd
data = pd.read_csv("compas-scores-two-years.csv")
data = data[data["days_b_screening_arrest"] <= 30]
data = data[data["days_b_screening_arrest"] >= -30]
#display(data.columns)

In [ ]:
#b)
#Finner de som ble klassifisert som høy eller lav risiko:
High_risk = data[data["score_text"].isin(["Medium","High"])]
Low_risk = data[data["score_text"] == "Low"]

#Finner hvor bra COMPAS gjorde det:
Falsk_positiv = High_risk[High_risk["two_year_recid"] == 0]
Sann_positiv = High_risk[High_risk["two_year_recid"] >= 1]
Falsk_negativ = Low_risk[Low_risk["two_year_recid"] >= 1]
Sann_negativ = Low_risk[Low_risk["two_year_recid"] == 0]

#Finner estimerte sannsynlihgetsverdier:
p_Falsk_positiv = Falsk_positiv.size/High_risk.size
p_Sann_positiv = Sann_positiv.size/High_risk.size
p_Falsk_negativ = Falsk_negativ.size/Low_risk.size
p_Sann_negativ = Sann_negativ.size/Low_risk.size

#Printer sannsynlighetsverdiene:
print("falsk positiv: ",p_Falsk_positiv)
print("sann positiv: ",p_Sann_positiv)
print()
print("falsk negativ: ",p_Falsk_negativ)
print("sann negativ: ",p_Sann_negativ)

falsk positiv:  0.37004725554343876
sann positiv:  0.6299527444565612

falsk negativ:  0.31452791581408945
sann negativ:  0.6854720841859105
